---
## Code Introduction

### Overview
This code performs the following tasks:

1. **Extracts Job Links**: Scrapes job links from LinkedIn and Glassdoor.
2. **Web Crawling**: Utilizes Selenium to crawl and gather information from the extracted links.
3. **Job Description Extraction**: Leverages ChatGPT 3.5 API to extract job descriptions.
4. **Data Sorting and Saving**: Sorts the collected data and saves it to a spreadsheet.

---


In [ ]:
from exchangelib import DELEGATE, Account, Credentials, Folder
import requests
from bs4 import BeautifulSoup
import openai
import pandas as pd
import time
import openpyxl
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import regex as re
# Initialize OpenAI API key
openai.api_key = ""

In [ ]:
def extract_job_details_from_text(text):
    # Initialize OpenAI API key (use environment variable or directly input your API key)

    # Define the prompt to extract information
    prompt = f"From the following job summary, use bullet points if possible, extract details like 'Job Title', 'Company Name', 'Job Link', 'Job Location', 'Salary', 'Number of Applicants', 'Posted Days Ago', 'Job Diploma Requirement', 'Job Years of Experience Requirement', 'Job Description':\n{text}\nDetails:"
    
    # Make the API call with GPT-3.5 Turbo
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant specialized in extracting job details."},
            {"role": "user", "content": prompt}
        ]
    )
    
    # Get the generated text
    details = response['choices'][0]['message']['content'].strip()
    
    return details


def job_cralwer(url, plat='Linkedin'):
    # Set a timeout of 10 seconds
    driver.set_page_load_timeout(10)
    
    # Load a URL
    try:
        driver.get(url)
    except TimeoutException:
        print("Timed out waiting for the page to load")
        return None
        
    job_info = ""
    if plat=='Linkedin':
        classes = ['jobs-unified-top-card__primary-description','jobs-company__box', 'jobs-description__container']
        # Expand job description
        time.sleep(3)
        driver.find_element(By.XPATH, '//span[text()="See more"]').click()
        # Loop through each class name to find the elements, get text, and append it to joined_text
        for class_name in classes:
            xpath_expression = f"//*[contains(@class, '{class_name}')]"
            elements = driver.find_elements(By.XPATH, xpath_expression)
            
            for element in elements:
                job_info += element.text + " "
        return job_info
    elif plat == 'Glassdoor':
        job_info_expression = f'//*[@id="JobDescriptionContainer"]'
        job_header_expression = f'//div[@class="d-flex"]'
        for xpath_exp in [job_info_expression, job_header_expression]:
            elements = driver.find_elements(By.XPATH, xpath_exp)
        
            for element in elements:
                job_info += element.text + " "
            
        return job_info



def parse_job_string(job_str):
    job_dict = {}
    description_started = False
    description_list = []
    others_list = []
    others_started = False

    for line in job_str.split('\n'):
        # Handle the 'Job Description' section
        if description_started:
            if re.match(r'\s+-', line):  # Bullet points in description
                description_list.append(line.strip('- '))
                continue
            else:  # Switch to 'Others' section
                description_started = False
                others_started = True

        # Handle the 'Others' section
        if others_started:
            others_list.append(line.strip('- '))
            continue

        # Handle other key-value pairs
        if ': ' in line or line.strip() == "- Job Description:":
            key, value = line.split(': ', 1) if ': ' in line else (line.strip('- :'), None)
            key = key.lstrip('- ').rstrip()  # Remove leading dashes and spaces
            if key == 'Job Description':
                description_started = True
            job_dict[key] = value if value else []

    job_dict['Job Description'] = "- "+"\n- ".join(description_list)
    job_dict['Others'] = None if not others_list else '\n'.join(others_list)
    job_dict['Queried Date'] = date.today()
    return job_dict


In [ ]:
from exchangelib import DELEGATE, Account, Credentials, Configuration
from datetime import datetime, timedelta
import pytz
from exchangelib import Credentials, Account, Configuration
from bs4 import BeautifulSoup

# Define your credentials
credentials = Credentials(username='', password='')
config = Configuration(server='outlook.office365.com', credentials=credentials)

# Connect to the account
account = Account(
    primary_smtp_address='', 
    config=config,
    autodiscover=False, 
    access_type=DELEGATE
)

# Access the inbox
inbox = account.inbox


# Delete after using
# Get the most recent three emails
recent_emails = account.inbox.all().order_by('-datetime_received')[:3]

for item in recent_emails:
    # Assuming 'body' contains the email body in HTML format
    email_body = item.body

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(email_body, 'html.parser')

    # Extract all href attributes
    all_hrefs = [a['href'] for a in soup.find_all('a', href=True)]
account.protocol.close()

# Your timezone
local_tz = pytz.timezone('America/New_York')

# Date and time for filtering emails received today, made timezone-aware
start_time = local_tz.localize(datetime.now().replace(hour=0, minute=0, second=0, microsecond=0))
end_time = local_tz.localize(datetime.now().replace(hour=23, minute=59, second=59, microsecond=999999))

job_urls = {
    'Linkedin' : [],
    'Glassdoor' : []
}
clean_job_urls = job_urls.copy()
linkedin_job_urls_list = []

# Loop through unread emails in the inbox received today
for item in inbox.filter(is_read=True, datetime_received__range=(start_time, end_time)):
    # Assuming 'body' contains the email body in HTML format
    email_body = item.body

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(email_body, 'html.parser')

    # Extract all href attributes
    all_hrefs = [a['href'] for a in soup.find_all('a', href=True)]
    
    # Filter out LinkedIn job URLs
    linkedin_job_urls = [url for url in all_hrefs if 'jobs/view/' in url]
    gd_job_urls = [html.unescape(url) for url in all_hrefs if 'jobListing' in url]
    
    if linkedin_job_urls:
        # Append these URLs to the list
        job_urls['Linkedin'].extend(linkedin_job_urls)
    if gd_job_urls:
        job_urls['Glassdoor'].extend(gd_job_urls)
        
job_urls['Linkedin'] = list(set(job_urls['Linkedin']))
clean_job_urls['Linkedin'] = [url.split("?")[0] for url in job_urls['Linkedin']]
clean_job_urls['Glassdoor'] = job_urls['Glassdoor']

## Linkedin

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep

options = webdriver.EdgeOptions()
options.add_argument("Application Support/Microsoft Edge")
options.add_argument("--profile-directory=Profile 1")
# Initialize the WebDriver for Edge
driver = webdriver.Edge(options=options)

In [ ]:
driver.get('https://www.linkedin.com/')
time.sleep(1.2)
driver.find_element(By.XPATH,'//input[@id="session_key"]').send_keys('')
driver.find_element(By.XPATH,'//input[@id="session_password"]').send_keys('')
driver.find_element(By.XPATH,'//button[@data-id="sign-in-form__submit-btn"]').click()

In [ ]:
df = pd.read_excel('JobHost.xlsx')
for url in clean_job_urls['Linkedin']:
    # Check if the URL already exists in the DataFrame
    if url not in df['Job Link'].values:
        print(url+' Not Saved')

In [ ]:
for url in clean_job_urls['Linkedin']:
    # Check if the URL already exists in the DataFrame
    if url not in df['Job Link'].values:
        # If it doesn't exist, scrape the job details
        job_description = job_cralwer(url)
        
        # Add the URL to the job description
        job_description = f'- Link:{url}\n' + job_description
        gpt_returns = extract_job_details_from_text(job_description)
        # Add the job details to the DataFrame
        job_details_json = parse_job_string(gpt_returns)
        df_new = pd.DataFrame([job_details_json])
        df = pd.concat([df, df_new], ignore_index=True)
        time.sleep(1)
        driver.get('https://www.linkedin.com/feed/')
        time.sleep(1)

In [ ]:
df.to_excel('JobHost.xlsx', index=False)

## Glassdoor

In [ ]:
df = pd.read_excel('GlassdoorJobPool.xlsx')
new_urls = [ url for url in clean_job_urls['Glassdoor'] if url not in df['Job Link'].values]
print(len(new_urls))

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep

options = webdriver.EdgeOptions()
options.add_argument("Application Support/Microsoft Edge")
options.add_argument("--profile-directory=Profile 1")
# Initialize the WebDriver for Edge
driver = webdriver.Edge(options=options)

In [ ]:
for url in new_urls[::3]:
    # Check if the URL already exists in the DataFrame
    if url not in df['Job Link'].values:
        # If it doesn't exist, scrape the job details
        job_description = job_cralwer(url, plat='Glassdoor')
        
        # Add the URL to the job description
        job_description = f'- Link:{url}\n' + job_description
        gpt_returns = extract_job_details_from_text(job_description)
        # Add the job details to the DataFrame
        job_details_json = parse_job_string(gpt_returns)
        df_new = pd.DataFrame([job_details_json])
        df = pd.concat([df, df_new], ignore_index=True)
        time.sleep(1)
        driver.get('https://google.com')
        time.sleep(1)

In [ ]:
df.to_excel('GlassdoorJobPool.xlsx', index=False)